In [ ]:
import torch
import random

In [ ]:
def linear(X,w,b):
    return torch.matmul(X,w) + b

def loss(y_hat,y):
    return (y_hat - y.reshape(y_hat.shape))**2/2

def sgd(params,lr,batch_size):
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()

def sqr(X):
    return X**2

In [ ]:
def synthetic_data(w, b, num_examples):  #@save
    """生成y=Xw+b+噪声"""
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [ ]:
from d2l import torch as d2l

In [ ]:
d2l.set_figsize()
d2l.plt.scatter(features[:, (1)].detach().numpy(), labels.detach().numpy(), 1);

In [ ]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # 这些样本是随机读取的，没有特定的顺序
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(
            indices[i: min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]


In [ ]:
lr = 0.03
epoch_num = 3
net = linear
batch_size = 10

w = torch.normal(0, 0.01, size=(2,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

for epoch in range(epoch_num):
    for X,y in data_iter(batch_size,features,labels):
        l = loss(net(X,w,b),y)
        l.sum().backward()
        sgd([w,b],lr,batch_size)
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')        
        

In [ ]:
w.detach(),true_w

In [ ]:
net = torch.nn.Sequential(torch.nn.Linear(2,1))

In [ ]:
net[0].weight.data.normal_(0,0.01)
net[0].bias.data.fill_(0)
print(net[0].weight.data)

In [ ]:
trainer = torch.optim.SGD(net.parameters(),0.01)
loss = torch.nn.MSELoss()

In [ ]:
from torch.utils.data import DataLoader,TensorDataset

In [ ]:
def load_array(data_arrays,batch_size,is_train=True):
    dataset = TensorDataset(*data_arrays)
    return DataLoader(dataset,batch_size,shuffle=is_train)

In [ ]:
data_iter = load_array((features,labels),batch_size)

In [ ]:
def train(net,optim,loss,data_iter,epochs):
    for epoch in range(epochs):
        for X,y in data_iter:
            l = loss(net(X),y)
            trainer.zero_grad()
            l.backward()
            trainer.step()
        with torch.no_grad():
            train_l = loss(net(features), labels)
            print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

In [ ]:
train(net,trainer,loss,data_iter,3)

In [ ]:
net[0].weight.data,true_w